##### Task 4: Interactive Chat Interface

This notebook demonstrates the creation of an interactive chat interface for the Financial Complaint Analysis RAG system using both Gradio and Streamlit.

##### Setup and Imports

In [29]:
# Setup paths and imports
import sys
import os
from pathlib import Path

# Add parent directory to path for imports
parent_dir = Path().absolute().parent
sys.path.append(str(parent_dir / 'src'))

print(f"📁 Working directory: {os.getcwd()}")
print(f"📁 Parent directory: {parent_dir}")

# Check system components
vector_store_path = parent_dir / 'vector_store'
src_path = parent_dir / 'src'

print(f"✅ Vector store exists: {vector_store_path.exists()}")
print(f"✅ Source directory exists: {src_path.exists()}")

if src_path.exists():
    src_files = [f.name for f in src_path.glob('*.py')]
    print(f"📁 Python files in src/: {src_files}")

📁 Working directory: d:\Coding\10Academy\Intelligent-Complaint-Analysis-for-Financial-Services-Week6\notebooks
📁 Parent directory: d:\Coding\10Academy\Intelligent-Complaint-Analysis-for-Financial-Services-Week6
✅ Vector store exists: True
✅ Source directory exists: True
📁 Python files in src/: ['chat_interface.py', 'config_manager.py', 'eda_preprocessing.py', 'embedding_indexer.py', 'main.py', 'rag_evaluator.py', 'rag_pipeline.py', 'streamlit_app.py', 'system_monitor.py', 'ui_app.py', 'ui_components.py', 'vector_store_utils.py', '__init__.py']


In [30]:
# Import required libraries
import time
import json
import logging
from typing import List, Dict, Optional, Tuple

try:
    import gradio as gr
    print("✅ Gradio imported successfully")
except ImportError:
    print("❌ Gradio not available. Install with: pip install gradio")
    gr = None

try:
    import streamlit as st
    print("✅ Streamlit imported successfully")
except ImportError:
    print("❌ Streamlit not available. Install with: pip install streamlit")
    st = None

import pandas as pd
print("✅ Core libraries imported successfully")

✅ Gradio imported successfully
✅ Streamlit imported successfully
✅ Core libraries imported successfully


##### Load RAG System Components

In [31]:
# Import our RAG system components
try:
    from rag_pipeline import create_simple_pipeline, RAGPipeline
    from chat_interface import ChatInterface, GradioApp, create_chat_app
    from vector_store_utils import ComplaintVectorStore
    print("✅ RAG system components imported successfully")
except ImportError as e:
    print(f"❌ Error importing RAG components: {e}")
    print("Make sure you've run the previous notebooks to create the necessary files")

✅ RAG system components imported successfully


In [32]:
# Initialize the RAG pipeline
try:
    vector_store_dir = str(parent_dir / 'vector_store')
    print(f"🔧 Initializing RAG pipeline with vector store: {vector_store_dir}")
    
    # Create the pipeline
    rag_pipeline = create_simple_pipeline(vector_store_dir)
    print("✅ RAG pipeline initialized successfully")
    
    # Test the pipeline
    test_query = "What are common credit card issues?"
    print(f"🧪 Testing with query: {test_query}")
    
    response = rag_pipeline.run(test_query)
    print(f"✅ Test successful!")
    print(f"   Answer length: {len(response.answer)} characters")
    print(f"   Sources retrieved: {len(response.retrieved_sources)}")
    print(f"   Confidence: {response.confidence_score:.3f}")
    print(f"   Processing time: {response.processing_time:.2f}s")
    
except Exception as e:
    print(f"❌ Error initializing RAG pipeline: {e}")
    print("Make sure you've run Task 2 to create the vector store")

🔧 Initializing RAG pipeline with vector store: d:\Coding\10Academy\Intelligent-Complaint-Analysis-for-Financial-Services-Week6\vector_store
Loading vector store components...


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:rag_pipeline:RAG Pipeline initialized successfully


Vector store loaded successfully!
- 930430 chunks
- 930430 vectors in index
- Embedding model: sentence-transformers/all-MiniLM-L6-v2
✅ RAG pipeline initialized successfully
🧪 Testing with query: What are common credit card issues?


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.56it/s]
ERROR:rag_pipeline:Error during retrieval: 'chunk_id'


✅ Test successful!
   Answer length: 65 characters
   Sources retrieved: 0
   Confidence: 0.000
   Processing time: 0.93s


##### Demo: Simple Chat Function

In [33]:
def simple_chat_demo():
    """Simple command-line chat demo"""
    print("\n🏦 CrediTrust Complaint Analysis Assistant")
    print("=" * 50)
    print("Ask questions about financial complaints. Type 'quit' to exit.\n")
    
    while True:
        try:
            query = input("❓ Your question: ").strip()
            
            if query.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye!")
                break
            
            if not query:
                continue
            
            print("\n🤔 Thinking...")
            response = rag_pipeline.run(query)
            
            print(f"\n💡 **Answer:**")
            print(response.answer)
            
            print(f"\n📊 **Confidence:** {response.confidence_score:.1%}")
            print(f"⏱️  **Processing time:** {response.processing_time:.2f}s")
            
            if response.retrieved_sources:
                print(f"\n📚 **Sources ({len(response.retrieved_sources)}):**")
                for i, source in enumerate(response.retrieved_sources[:3], 1):
                    meta = source.metadata
                    print(f"   {i}. Product: {meta.get('product', 'Unknown')} | "
                          f"Category: {meta.get('category', 'Unknown')} | "
                          f"Score: {source.score:.3f}")
                    print(f"      Text: {source.text[:100]}...")
            
            print("\n" + "-" * 50)
            
        except KeyboardInterrupt:
            print("\n\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")

# Uncomment to run the demo
# simple_chat_demo()

##### Gradio Interface

In [34]:
# Create and launch Gradio interface
if gr is not None:
    try:
        print("🚀 Creating Gradio interface...")
        
        # Create the chat app
        chat_app = create_chat_app(
            vector_store_dir=vector_store_dir,
            enable_streaming=False,
            share=False
        )
        
        print("✅ Gradio app created successfully!")
        print("\n📝 To launch the interface, run:")
        print("   chat_app.launch()")
        print("\n🌐 Or run the standalone app:")
        print("   python ../src/chat_interface.py")
        
    except Exception as e:
        print(f"❌ Error creating Gradio interface: {e}")
else:
    print("⚠️  Gradio not available. Install with: pip install gradio")

🚀 Creating Gradio interface...
✅ Gradio app created successfully!

📝 To launch the interface, run:
   chat_app.launch()

🌐 Or run the standalone app:
   python ../src/chat_interface.py


In [35]:
# Launch Gradio interface (uncomment to run)
# if 'chat_app' in locals():
#     print("🚀 Launching Gradio interface...")
#     chat_app.launch(
#         share=False,
#         server_port=7860,
#         show_error=True
#     )

##### Streamlit Interface Demo

In [37]:
# Display Streamlit app code
streamlit_app_code = '''
import streamlit as st
import sys
from pathlib import Path

# Add src to path
sys.path.append(str(Path(__file__).parent))

from rag_pipeline import create_simple_pipeline
from chat_interface import ChatInterface

# Initialize session state
if "rag_pipeline" not in st.session_state:
    st.session_state.rag_pipeline = create_simple_pipeline("../vector_store")
    st.session_state.chat_interface = ChatInterface(st.session_state.rag_pipeline)

if "messages" not in st.session_state:
    st.session_state.messages = []

# App header
st.title("🏦 CrediTrust Complaint Analysis Assistant")
st.markdown("Ask questions about financial complaints and get AI-powered insights.")

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("Ask about financial complaints..."):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Get AI response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = st.session_state.rag_pipeline.run(prompt)
            st.markdown(response.answer)
            
            # Show sources
            if response.retrieved_sources:
                with st.expander(f"📚 Sources ({len(response.retrieved_sources)})"):
                    for i, source in enumerate(response.retrieved_sources, 1):
                        st.markdown(f"**Source {i}** (Score: {source.score:.3f})")
                        meta = source.metadata
                        st.markdown(f"Product: {meta.get('product', 'Unknown')} | "
                                  f"Category: {meta.get('category', 'Unknown')}")
                        st.markdown(f"> {source.text[:200]}...")
                        st.markdown("---")
    
    # Add assistant message
    st.session_state.messages.append({"role": "assistant", "content": response.answer})

# Sidebar
with st.sidebar:
    st.header("💡 Tips")
    st.markdown("""
    - Be specific in your questions
    - Ask about products, issues, or trends
    - Check the sources for verification
    """)
    
    if st.button("Clear Chat"):
        st.session_state.messages = []
        st.rerun()
'''

print("📝 Streamlit app code:")
print(streamlit_app_code)

print("\n🚀 To run the Streamlit app:")
print("   streamlit run ../src/streamlit_app.py")

📝 Streamlit app code:

import streamlit as st
import sys
from pathlib import Path

# Add src to path
sys.path.append(str(Path(__file__).parent))

from rag_pipeline import create_simple_pipeline
from chat_interface import ChatInterface

# Initialize session state
if "rag_pipeline" not in st.session_state:
    st.session_state.rag_pipeline = create_simple_pipeline("../vector_store")
    st.session_state.chat_interface = ChatInterface(st.session_state.rag_pipeline)

if "messages" not in st.session_state:
    st.session_state.messages = []

# App header
st.title("🏦 CrediTrust Complaint Analysis Assistant")
st.markdown("Ask questions about financial complaints and get AI-powered insights.")

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("Ask about financial complaints..."):
    # Add user message
    st.session_state.messages.append({"role": "user",

##### Testing and Validation

In [38]:
# Test various queries to validate the system
test_queries = [
    "What are the most common credit card issues?",
    "Show me complaints about unauthorized charges",
    "What billing problems do customers face?",
    "Tell me about mortgage-related complaints",
    "What are common debt collection issues?"
]

print("🧪 Testing the RAG system with various queries...")
print("=" * 60)

for i, query in enumerate(test_queries, 1):
    try:
        print(f"\n{i}. Query: {query}")
        response = rag_pipeline.run(query)
        
        print(f"   ✅ Success!")
        print(f"   📊 Confidence: {response.confidence_score:.1%}")
        print(f"   📚 Sources: {len(response.retrieved_sources)}")
        print(f"   ⏱️  Time: {response.processing_time:.2f}s")
        print(f"   📝 Answer length: {len(response.answer)} chars")
        
        # Show first few words of answer
        answer_preview = ' '.join(response.answer.split()[:15])
        print(f"   💡 Preview: {answer_preview}...")
        
    except Exception as e:
        print(f"   ❌ Error: {e}")

print("\n✅ Testing completed!")

🧪 Testing the RAG system with various queries...

1. Query: What are the most common credit card issues?


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.21it/s]
ERROR:rag_pipeline:Error during retrieval: 'chunk_id'


   ✅ Success!
   📊 Confidence: 0.0%
   📚 Sources: 0
   ⏱️  Time: 0.56s
   📝 Answer length: 65 chars
   💡 Preview: I couldn't find any relevant information to answer your question....

2. Query: Show me complaints about unauthorized charges


Batches: 100%|██████████| 1/1 [00:00<00:00, 34.66it/s]
ERROR:rag_pipeline:Error during retrieval: 'chunk_id'


   ✅ Success!
   📊 Confidence: 0.0%
   📚 Sources: 0
   ⏱️  Time: 0.33s
   📝 Answer length: 65 chars
   💡 Preview: I couldn't find any relevant information to answer your question....

3. Query: What billing problems do customers face?


...

   ✅ Success!
   📊 Confidence: 0.0%
   📚 Sources: 0
   ⏱️  Time: 0.40s
   📝 Answer length: 65 chars
   💡 Preview: I couldn't find any relevant information to answer your question....

4. Query: Tell me about mortgage-related complaints


Batches: 100%|██████████| 1/1 [00:00<00:00, 43.40it/s]
ERROR:rag_pipeline:Error during retrieval: 'chunk_id'


   ✅ Success!
   📊 Confidence: 0.0%
   📚 Sources: 0
   ⏱️  Time: 0.30s
   📝 Answer length: 65 chars
   💡 Preview: I couldn't find any relevant information to answer your question....

5. Query: What are common debt collection issues?


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.03it/s]
ERROR:rag_pipeline:Error during retrieval: 'chunk_id'


   ✅ Success!
   📊 Confidence: 0.0%
   📚 Sources: 0
   ⏱️  Time: 0.36s
   📝 Answer length: 65 chars
   💡 Preview: I couldn't find any relevant information to answer your question....

✅ Testing completed!


##### Summary and Next Steps

##### What We've Built:

1. **Modular Chat Interface**: Object-oriented design with separate components for:
   - Chat session management
   - Response formatting
   - Streaming support
   - Interface controllers

2. **Gradio Web Interface**: Professional chat interface with:
   - Real-time conversation
   - Source display and verification
   - Confidence indicators
   - Session management
   - Export functionality

3. **Streamlit Alternative**: Clean, modern interface with:
   - Native chat components
   - Expandable source sections
   - Sidebar controls
   - Session state management

##### Key Features Implemented:

✅ **Text input box** for user questions
✅ **Submit/Ask button** for query processing
✅ **Display area** for AI-generated answers
✅ **Source display** with metadata and similarity scores
✅ **Clear button** to reset conversations
✅ **Confidence indicators** for answer reliability
✅ **Processing time** display
✅ **Error handling** and user feedback
✅ **Modular architecture** for easy maintenance

##### How to Run:

1. **Gradio Interface**:
   ```bash
   python src/chat_interface.py
   ```

2. **Streamlit Interface**:
   ```bash
   streamlit run src/streamlit_app.py
   ```

3. **Main Application Launcher**:
   ```bash
   python app.py web
   ```

##### Files Created:

- `src/chat_interface.py` - Main Gradio interface with modular components
- `src/streamlit_app.py` - Streamlit alternative interface
- `src/ui_components.py` - Reusable UI components
- `app.py` - Updated main launcher with web interface support

The interfaces provide a user-friendly way for non-technical users to interact with the RAG system while maintaining transparency through source display and confidence indicators.